In [ ]:
############################################################
# Proportion Ranking of Proteins by Detection Rate
# Author: Alexa Canchola
# Advisor: Wei-Chun Chou
# Date: Jul 22, 2025
############################################################

In [ ]:
############################################################
# 1.  Load libraries ---------------------------------------
############################################################
if (!requireNamespace("readxl", quietly = TRUE)) install.packages("readxl")
if (!requireNamespace("writexl", quietly = TRUE)) install.packages("writexl")

library(readxl)
library(dplyr)
############################################################
# 2.  Read the dataset -------------------------------------
############################################################
file_path <- "Dataset-for-protein-freq-analysis-v2.xlsx"  # adjust if needed
df <- read_excel(file_path, sheet = 1)   # 'Sheet2' is sheet 1 in zero-based indexing

############################################################
# 3.  Identify protein columns -----------------------------
#     (assumes first two cols are NP and study IDs)
############################################################
protein_cols <- colnames(df)[-(1:2)]     # every column after 2

############################################################
# 4.  Compute detection rate -------------------------------
############################################################
det_rate <- df %>%
  mutate(across(all_of(protein_cols), ~ as.integer(.x > 0))) %>%  # 1 = detected
  summarise(across(all_of(protein_cols), mean, na.rm = TRUE)) %>%
  tidyr::pivot_longer(everything(),
                      names_to = "protein",
                      values_to = "detection_rate") %>%
  arrange(desc(detection_rate))

############################################################
# 5.  Show top 10 proteins ---------------------------------
############################################################
print(head(det_rate, 20))

############################################################
# 6.  (Optional) export full table -------------------------
############################################################
write.csv(det_rate, "/content/protein_detection_rates.csv", row.names = FALSE)
# In Colab you can download the CSV with:
# from google.colab import files
# files.download('/content/protein_detection_rates.csv')

############################################################
# 7.  Visualise the top 20 proteins ------------------------
############################################################
if (!requireNamespace("ggplot2", quietly = TRUE)) install.packages("ggplot2")
if (!requireNamespace("scales",   quietly = TRUE)) install.packages("scales")

library(ggplot2)
library(scales)

top20 <- det_rate %>%
  slice_max(detection_rate, n = 20)      # keep 20 highest rates

ggplot(top20, aes(x = reorder(protein, detection_rate),
                  y = detection_rate)) +
  geom_col(fill = "steelblue") +
  coord_flip() +
  scale_y_continuous(labels = percent_format(accuracy = 1)) +
  labs(title = "Top 20 most frequently detected proteins",
       x = "Protein",
       y = "Detection proportion") +
  theme_minimal(base_size = 12)